In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
PATH = '/kaggle/input/tabular-playground-series-jun-2021/'
train_path = os.path.join(PATH,'train.csv')
test_path = os.path.join(PATH,'test.csv')
sub = os.path.join(PATH,'sample_submission.csv')

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
train_df.head()

In [ ]:
y = train_df['target']
X = train_df.drop(columns= ['id','target'],axis=1)
X

In [ ]:

mapper = {'Class_1':1,'Class_2':2,'Class_3':3,'Class_4':4,'Class_5':5
         ,'Class_6':6,'Class_7':7,'Class_8':8,'Class_9':9}

y_cat = y.copy()
y.replace(mapper,inplace=True)

In [ ]:
# y_test = test_df['target']
test_x = test_df.drop(columns= ['id'],axis=1)
test_x

In [ ]:
# from sklearn.model_selection import train_test_split

# train_x, test_x, train_y, test_y = train_test_split(X, y, random_state = 9 , stratify = y,)
# train_x.shape,train_y.shape,test_x.shape,test_y.shape

In [ ]:
train_x  = X
train_y= y

### Model training and Stacking

In [ ]:
def model_predictions(model, train_x, train_y, test_x,name):
    #train the model
    
    # we are just storing predictions not probabilities if decision tree because 
    # dt only gives boolean when we use predcit_proba
    if name == "dt":
        model.fit(train_x,train_y)   
        
        #storing predictions for train and test
        pred_train=model.predict(train_x)
        pred_test=model.predict(test_x)
    else:
        model.fit(train_x,train_y)      
        #storing predictions for train and test
        pred_train=model.predict_proba(train_x)
        pred_test=model.predict_proba(test_x)
    return pred_train, pred_test

In [ ]:
#Model 1 - Decision Tree
DT=DecisionTreeClassifier(random_state= 101)
M1_train, M1_test = model_predictions(DT, train_x, train_y, test_x,"dt")

In [ ]:
#Model 4 - Random Forest
RF=RandomForestClassifier(random_state= 101)
M4_train, M4_test = model_predictions(RF, train_x, train_y, test_x,"rf")

In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X=train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
#Model 2 - Logistic Regression
LR=LogisticRegression(random_state= 101,solver='saga')
M2_train, M2_test = model_predictions(LR, train_x, train_y, test_x,"lg")

In [ ]:
# #Model 3 - k Nearest Neighbour
# knn=KNeighborsClassifier(n_neighbors=9)
# M3_train, M3_test = model_predictions(knn, train_x, train_y, test_x,"kn")

In [ ]:
a1 = pd.DataFrame(data = M1_train)
a2 = pd.DataFrame(data = M2_train)
# a3 = pd.DataFrame(data = M3_train)
a4 = pd.DataFrame(data = M4_train)

In [ ]:
train_prediction = pd.concat([a1,a2,a4],axis=1,ignore_index=True)
train_prediction

In [ ]:
a1 = pd.DataFrame(data = M1_test)
a2 = pd.DataFrame(data = M2_test)
# a3 = pd.DataFrame(data = M3_test)
a4 = pd.DataFrame(data = M4_test)
test_prediction = pd.concat([a1,a2,a4],axis=1,ignore_index=True)
test_prediction

In [ ]:
# Stacker Model
from sklearn.svm import SVC
model = SVC(probability=True)
model.fit(train_prediction, train_y)

y_pred_prob = model.predict_proba(test_prediction)
y_pred = model.predict(test_prediction)
y_pred
# model.score(test_prediction,test_y)

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_pred, y_pred_prob)

In [ ]:
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
ids = test_df['id']
test_df = test_df.drop(columns=['id'])
# sc.transform(test_df)
# test_df = pca.transform(test_df)
print(test_df.shape)

In [ ]:
col_names = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 
             'Class_7', 'Class_8', 'Class_9']
df_sub = pd.DataFrame(data=y_pred_prob, columns=col_names)
df_sub.shape

In [ ]:
df_ids = pd.DataFrame(data=ids, columns=['id'])
print(df_ids.shape)


df_final = pd.concat([df_ids, df_sub], axis=1)
df_final.head()

In [ ]:
df_final.to_csv('submission.csv', index=False)